In [1]:
import numpy as np

def value_iteration_grid_world():
    # --- 1. SETUP THE ENVIRONMENT ---
    grid_size = 4
    gamma = 1.0  # Discount factor (1.0 for shortest path)
    theta = 1e-4 # Convergence threshold

    # Terminal states (Top-Left and Bottom-Right)
    terminal_states = [0, 15]
    actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']

    # Initialize Value Function V(s) to zeros
    V = np.zeros(grid_size * grid_size)

    print("Starting Value Iteration...")
    iteration = 0

    # --- 2. THE ALGORITHM (Value Iteration) ---
    while True:
        delta = 0
        # Create a copy for synchronous updates
        V_new = np.copy(V)

        # Loop over every state in the world
        for s in range(grid_size * grid_size):
            if s in terminal_states:
                continue # Value of terminal state is always 0

            # Calculate coordinates
            row, col = divmod(s, grid_size)

            # Find the max value among all possible actions
            action_values = []

            for action in actions:
                # --- MODEL LOGIC (Simulating the move) ---
                next_r, next_c = row, col

                if action == 'UP':    next_r = max(row - 1, 0)
                elif action == 'DOWN':  next_r = min(row + 1, grid_size - 1)
                elif action == 'LEFT':  next_c = max(col - 1, 0)
                elif action == 'RIGHT': next_c = min(col + 1, grid_size - 1)

                next_state = next_r * grid_size + next_c

                # Standard Reward is -1 per step
                reward = -1

                # Bellman Optimality Equation: R + gamma * V(s')
                val = reward + gamma * V[next_state]
                action_values.append(val)

            # Update V(s) with the BEST possible action (Greedy)
            best_value = max(action_values)
            V_new[s] = best_value

            # Check how much the value changed
            delta = max(delta, abs(best_value - V[s]))

        V = V_new
        iteration += 1

        # Stop if converged
        if delta < theta:
            print(f"Converged after {iteration} iterations.")
            break

    # --- 3. DISPLAY RESULTS ---
    print("\nOptimal Value Function (V*):")
    print(np.round(V.reshape(4, 4), 1))

    # Optional: Display the Optimal Policy (Arrows)
    print("\nOptimal Policy (Planning Result):")
    arrows = {0: '↑', 1: '↓', 2: '←', 3: '→'}
    policy_grid = []

    for s in range(grid_size * grid_size):
        if s in terminal_states:
            policy_grid.append(" T ")
            continue

        row, col = divmod(s, grid_size)
        best_action_idx = -1
        best_val = -float('inf')

        # Check neighbors again to find which one gave that best value
        for i, action in enumerate(actions):
            next_r, next_c = row, col
            if action == 'UP':    next_r = max(row - 1, 0)
            elif action == 'DOWN':  next_r = min(row + 1, grid_size - 1)
            elif action == 'LEFT':  next_c = max(col - 1, 0)
            elif action == 'RIGHT': next_c = min(col + 1, grid_size - 1)

            val = -1 + gamma * V[next_r * grid_size + next_c]
            if val > best_val:
                best_val = val
                best_action_idx = i

        policy_grid.append(f" {arrows[best_action_idx]} ")

    # Print Policy Grid
    print("-" * 17)
    for i in range(0, 16, 4):
        print("|".join(policy_grid[i:i+4]))
        print("-" * 17)

if __name__ == "__main__":
    value_iteration_grid_world()

Starting Value Iteration...
Converged after 4 iterations.

Optimal Value Function (V*):
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Optimal Policy (Planning Result):
-----------------
 T | ← | ← | ↓ 
-----------------
 ↑ | ↑ | ↑ | ↓ 
-----------------
 ↑ | ↑ | ↓ | ↓ 
-----------------
 ↑ | → | → | T 
-----------------


This Python code implements the **Value Iteration algorithm** to find the optimal value function and policy for a simple **Grid World** environment, a classic problem in Reinforcement Learning.

Here's a breakdown of the code:

1.  **Environment Setup (`--- 1. SETUP THE ENVIRONMENT ---`)**:
    *   `grid_size = 4`: Defines a 4x4 grid, resulting in 16 possible states (0 to 15).
    *   `gamma = 1.0`: This is the **discount factor**. A value of 1.0 means future rewards are valued equally to immediate rewards, which is typical for shortest path problems where the goal is to reach a terminal state with minimum steps.
    *   `theta = 1e-4`: The **convergence threshold**. The algorithm stops when the maximum change in the value function (`delta`) between iterations falls below this small value, indicating that the values have stabilized.
    *   `terminal_states = [0, 15]`: These are the states (top-left and bottom-right corners) where the agent receives a reward and the episode ends. Their value is fixed at 0.
    *   `actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']`: The possible actions an agent can take from any non-terminal state.
    *   `V = np.zeros(grid_size * grid_size)`: Initializes the **value function** `V(s)` for all states `s` to zero. This `V` array will eventually store the maximum expected cumulative reward for reaching a terminal state from each state, following an optimal policy.

2.  **The Algorithm (Value Iteration) (`--- 2. THE ALGORITHM (Value Iteration) ---`)**:
    *   The `while True:` loop continues iteratively until the value function converges.
    *   `delta = 0`: This variable tracks the largest change in any state's value during an iteration. It's used to check for convergence.
    *   `V_new = np.copy(V)`: A copy of the current value function is made for synchronous updates. This means that all calculations within an iteration use the values from the *previous* iteration, preventing a state update from immediately affecting the value calculation for another state in the same iteration.
    *   **State Iteration**: The code iterates through each state `s` in the grid.
        *   If `s` is a `terminal_state`, its value is already 0, so it's skipped.
        *   For non-terminal states, it calculates `row` and `col` coordinates from the state index.
    *   **Action Value Calculation**: For each state, it considers all possible actions (`UP`, `DOWN`, `LEFT`, `RIGHT`).
        *   `next_r, next_c`: Calculates the coordinates of the next state after taking an action. Boundary conditions (e.g., hitting a wall) are handled by `max(row - 1, 0)` or `min(row + 1, grid_size - 1)` to keep the agent within the grid.
        *   `next_state`: Converts the `next_r, next_c` back to a single state index.
        *   `reward = -1`: A standard setup where each step taken incurs a cost (negative reward) of -1. The goal is often to minimize steps to a terminal state.
        *   **Bellman Optimality Equation**: `val = reward + gamma * V[next_state]` calculates the value of taking a specific action from the current state. This equation is central to Value Iteration: the value of a state via a specific action is the immediate reward received plus the discounted value of the resulting next state.
        *   `action_values.append(val)`: Stores the calculated value for each possible action.
    *   **Value Update**: `best_value = max(action_values)`: The value of the current state `s` is updated to the maximum possible value achievable by taking the *best* action from that state (`V_new[s] = best_value`). This greedy choice ensures that the algorithm moves towards an optimal policy.
    *   **Convergence Check**: `delta = max(delta, abs(best_value - V[s]))`: `delta` is updated to be the maximum absolute change in the value of any state during the current iteration. If `delta` falls below `theta`, it signifies that the value function has stabilized sufficiently, and the loop breaks.
    *   `V = V_new`: The updated values become the current values for the next iteration.

3.  **Display Results (`--- 3. DISPLAY RESULTS ---`)**:
    *   **Optimal Value Function (V*)**: After convergence, the final `V` array (which represents `V*`, the optimal value function) is printed, reshaped into the 4x4 grid format. Each number indicates the maximum expected reward (or minimum cost, as rewards are negative) for reaching a terminal state from that particular cell, assuming optimal actions are taken.
    *   **Optimal Policy (Planning Result)**:
        *   The optimal policy dictates which action to take in each state. It's derived directly from the converged `V*`.
        *   For each non-terminal state, the code again considers all possible actions. It calculates the value of taking each action (`-1 + gamma * V[next_state]`) and selects the action that leads to the highest value. This chosen action is the `best_action_idx`.
        *   `arrows`: A dictionary maps numeric action indices to directional arrow symbols (↑, ↓, ←, →).
        *   `policy_grid`: A list is constructed, where each element is an arrow representing the optimal action for that state, or ' T ' for terminal states.
        *   Finally, the `policy_grid` is printed in a 4x4 grid format, visually showing the optimal direction to move from each state to follow the optimal path to a terminal state.